# Status #

- [x] Outline
- [ ] Introduction
- [ ] Exercise 1
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 2
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 3
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 4
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Conclusion

# Introduction #

Run the cell below to set everything up!

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex6 import *

import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.layers.experimental.preprocessing as preprocessing

### 1) Explore Augmentation

Uncomment a transformation to see what it does. You can experiment with the parameter values too, if you like. (The `factor` parameters should be greater than 0 and, generally, less than 1.)

In [ ]:
preprocess = keras.Sequential([
    # preprocessing.RandomContrast(factor=0.5)
    preprocessing.RandomFlip(mode='horizontal'),
    # preprocessing.RandomHeight(factor=0.15),
    # preprocessing.RandomWidth(factor=0.15),
    # preprocessing.RandomRotation(factor=0.20),
    # preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
])

In [ ]:
# Run this cell to get credit for question 1!
q_1.check()

### 2) Choosing Augmentations

Consider this dataset. What might be some good augmentations?

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

Now consider this dataset. What might be some good augmentations?

### 3) Add Augmentations to Car Classifier


In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

### 4) Train Model

Now train your model.


Did your choice of augmentations decrease the training loss? Why or why not?

In [ ]:

q_1.check()

In [ ]:

q_1.assert_check()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

# Conclusion #


# The End #

That's all for this course!